# Data / Feature Engineering

In this notebook, we acheive the following feature engineering efforts:
    - use image processing library to parse the image into matrices
    - we reshape the matrice 
    - store the matrice into table 

In [2]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging
from snowflake.snowpark.functions import *
from snowflake.snowpark import *

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['APP_DB']['snow_opt_wh']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,BECKY,PUBLIC,INDSOL_DICOM_DB,PUBLIC


In [ ]:
stmts = [
   f'''
    PUT file://{PROJECT_HOME_DIR}/data/train/PNEUMONIA/*.jpeg @data_stg/PNEUMONIA 
    auto_compress = false
    overwrite = true;
    '''
    ,f''' alter stage data_stg refresh; 
    
    '''

    ,f'''
    PUT file://{PROJECT_HOME_DIR}/data/train/NORMAL/*.jpeg @data_stg/NORMAL 
    auto_compress = false
    overwrite = true;
    '''
    ,f''' alter stage data_stg refresh; '''

    ,f'''
    PUT file://../python/skimage_parser_fn.py @lib_stg/scripts 
    auto_compress = false
    overwrite = true;

    '''
    ,f'''
    create or replace function skimage_parser_fn(image_fl varchar)
     returns variant
     language python
     runtime_version = '3.8'
     packages = ('snowflake-snowpark-python','numpy', 'pandas', 'scikit-learn' ,'scikit-image')
     imports = ('@lib_stg/scripts/skimage_parser_fn.py')
     handler = 'skimage_parser_fn.main'
     ;
    '''
      
    ]

for stmt in stmts:
    sp_session.sql(stmt).collect()

In [19]:
stmts = [
   f'''
    
    PUT file://../python/skimage_parser_fn.py @lib_stg/scripts 
    auto_compress = false
    overwrite = true;

    '''
    ,f'''
    create or replace function skimage_parser_fn(image_fl varchar)
     returns variant
     language python
     runtime_version = '3.8'
     packages = ('snowflake-snowpark-python','numpy', 'pandas', 'scikit-learn' ,'scikit-image')
     imports = ('@lib_stg/scripts/skimage_parser_fn.py')
     handler = 'skimage_parser_fn.main'
     ;
    '''
      
    ]

for stmt in stmts:
    sp_session.sql(stmt).collect()




In [20]:
df = sp_session.sql('''
    select 
    relative_path
    ,concat('@data_stg/',relative_path) as full_image_path
    ,skimage_parser_fn(BUILD_SCOPED_FILE_URL(@data_stg,RELATIVE_PATH)) as parsed_image_info
from directory(@data_stg) 

''')
df.show(1)

------------------------------------------------------------------------------------------------------------------------------
|"RELATIVE_PATH"               |"FULL_IMAGE_PATH"                       |"PARSED_IMAGE_INFO"                                 |
------------------------------------------------------------------------------------------------------------------------------
|person374_bacteria_1710.jpeg  |@data_stg/person374_bacteria_1710.jpeg  |{                                                   |
|                              |                                        |  "image_array": "[54, 30, 26, 24, 26, 29, 29, ...  |
|                              |                                        |  "image_array_shape_0": 150,                       |
|                              |                                        |  "image_array_shape_1": 150,                       |
|                              |                                        |  "normalized_image_array": "[0.211764

In [18]:
df.to_pandas().PARSED_IMAGE_INFO.iloc[0]

KeyboardInterrupt: 

In [166]:
df.write.save_as_table('parsed_images', mode="overwrite" ,table_type='transient')

In [177]:
df2 = sp_session.table('parsed_images').select(\
col("FULL_IMAGE_PATH").cast(T.StringType()).alias('IMAGE_FILE_PATH'),\
split("RELATIVE_PATH",lit("/"))[0].cast(T.StringType()).alias('CLASS_LABEL'),\

col("PARSED_IMAGE_INFO")["image_array"].cast(T.VariantType()).alias('IMAGE_ARRAY'),\
col("PARSED_IMAGE_INFO")["image_array_shape_0"].cast(T.IntegerType()).alias('IMAGE_ARRAY_SHAPE_0'),\
col("PARSED_IMAGE_INFO")["image_array_shape_1"].cast(T.IntegerType()).alias('IMAGE_ARRAY_SHAPE_1'),\
col("PARSED_IMAGE_INFO")["normalized_image_array"].cast(T.VariantType()).alias('NORMALIZED_IMAGE_ARRAY'),\
col("PARSED_IMAGE_INFO")["parsing_execption"].cast(T.StringType()).alias('PARSING_EXECPTION'),\
col("PARSED_IMAGE_INFO")["parsing_status"].cast(T.BooleanType()).alias('STATUS'),\
col("PARSED_IMAGE_INFO")["resized_feature"].cast(T.VariantType()).alias('RESIZED_FEATURE')\
)
sp_session.createDataFrame(pd.DataFrame({"CLASS_LABEL_NUM":[0,1],"LABEL":['PNEUMONIA','NORMAL']})).write.save_as_table('LABELS', mode="overwrite" ,table_type='transient')

#
df2 = df2.join(sp_session.table('LABELS'),df2['CLASS_LABEL']==LABELS['LABEL'])
df2 = df2.drop('LABEL').create_or_replace_view('images_formatted')






sp_session.table('images_formatted').limit(5).to_pandas()


,IMAGE_FILE_PATH,CLASS_LABEL,IMAGE_ARRAY,IMAGE_ARRAY_SHAPE_0,IMAGE_ARRAY_SHAPE_1,NORMALIZED_IMAGE_ARRAY,PARSING_EXECPTION,STATUS,RESIZED_FEATURE,CLASS_LABEL_NUM
0,@data_stg/NORMAL/NORMAL2-IM-0863-0001.jpeg,NORMAL,"""[55, 65, 69, 75, 76, 87, 126, 128, 125, 122, ...",150,150,"""[0.21568627450980393, 0.2549019607843137, 0.2...",None,True,"""[0.21568627450980393, 0.2549019607843137, 0.2...",1
1,@data_stg/NORMAL/NORMAL2-IM-0890-0001.jpeg,NORMAL,"""[2, 73, 129, 137, 140, 144, 148, 140, 131, 13...",150,150,"""[0.00784313725490196, 0.28627450980392155, 0....",None,True,"""[0.00784313725490196, 0.28627450980392155, 0....",1
2,@data_stg/NORMAL/NORMAL2-IM-0865-0001.jpeg,NORMAL,"""[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9,...",150,150,"""[0.03137254901960784, 0.03137254901960784, 0....",None,True,"""[0.03137254901960784, 0.03137254901960784, 0....",1
3,@data_stg/NORMAL/NORMAL2-IM-0904-0001.jpeg,NORMAL,"""[65, 70, 68, 65, 60, 56, 52, 48, 45, 43, 42, ...",150,150,"""[0.2549019607843137, 0.27450980392156865, 0.2...",None,True,"""[0.2549019607843137, 0.27450980392156865, 0.2...",1
4,@data_stg/NORMAL/NORMAL2-IM-0887-0001.jpeg,NORMAL,"""[0, 0, 0, 6, 14, 22, 27, 27, 26, 28, 42, 61, ...",150,150,"""[0.0, 0.0, 0.0, 0.023529411764705882, 0.05490...",None,True,"""[0.0, 0.0, 0.0, 0.023529411764705882, 0.05490...",1


--- 
### Closeout

    With that we are finished this section of the demo setup

In [173]:
sp_session.close()
print('Finished!!!')

Finished!!!
